# Creating STAC Catalogs for Contextual Data

In [29]:
from pathlib import Path
from datetime import datetime

# pystac
from pystac import Catalog, Item, Asset, CatalogType

# dask_geopandas, dask-expr, pyogrio
import dask_geopandas as dg

import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import MultiPoint

## Create a STAC Catalog from scratch

### Create a new, empty catalog

In [30]:
catalog = Catalog(id="contextual", description="Contextual dataset")
path_root = Path("./data/catalog/")
catalog.normalize_and_save(
    root_href=path_root.absolute().resolve().as_posix(),
    catalog_type=CatalogType.SELF_CONTAINED,
)

### Make a STAC catalog for BAG data

In [31]:
path_BAG = Path('./data/bag.parquet')

In [32]:
data_BAG = dg.read_parquet(path_BAG)
data_BAG

,rdf_seealso,identificatie,bouwjaar,status,gebruiksdoel,oppervlakte_min,oppervlakte_max,aantal_verblijfsobjecten,geometry
npartitions=50,,,,,,,,,
,string,string,int64,string,string,float64,float64,int64,geometry
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [33]:
def get_metadata_bag(ddf):

    metadata = dict()
    
    # Metadata can be retrieved from the delayed object
    metadata["columns"] = ddf.columns.tolist()

    # Metadata can be retrieved from the first partition
    gs_part0 = ddf.get_partition(0).compute()
    metadata["epsg_code"] = gs_part0["geometry"].crs.to_epsg()
    
    # Metadata needs to be retrieved from all partitions
    # For now iterate over all partitions, improvement from dask-geopandas WIP 
    arr_bbox = np.empty((0, 4))
    list_geom_types = []
    for i in range(ddf.npartitions):
        gs = ddf.get_partition(i)["geometry"].compute()
        arr_bbox = np.vstack((arr_bbox, gs.total_bounds))
        list_geom_types = list_geom_types + gs.geom_type.unique().tolist()
    
    # Get the bounding box
    bbox = [
        arr_bbox[:, 0].min(),
        arr_bbox[:, 1].min(),
        arr_bbox[:, 2].max(),
        arr_bbox[:, 3].max(),
    ]

    # Get unique geometry types
    geom_types = list(set(list_geom_types))

    metadata["bbox"] = bbox
    metadata["geometry_types"] = geom_types

    return metadata

In [34]:
metadata = get_metadata_bag(data_BAG)
metadata

{'columns': ['rdf_seealso',
  'identificatie',
  'bouwjaar',
  'status',
  'gebruiksdoel',
  'oppervlakte_min',
  'oppervlakte_max',
  'aantal_verblijfsobjecten',
  'geometry'],
 'epsg_code': 28992,
 'bbox': [13603.33, 306900.396, 277924.306, 617112.488],
 'geometry_types': ['Polygon']}

In [35]:
# Create a new Item
item = Item(
    id="bag",
    geometry=None,
    bbox=metadata["bbox"],  # Bounding box of the dataset
    datetime=datetime.strptime(
        "2024-03-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"
    ),  # datatime of the dataset, from https://service.pdok.nl/lv/bag/atom/bag.xml
    properties={
        "columns": metadata["columns"],
        "geometry_types": metadata["geometry_types"],
        "reference_system": metadata["epsg_code"],
    },
)

# Add the Datacube Extension to the Item
item.stac_extensions.append("datacube")

# Add path to the BAG dataset as an asset
asset = Asset(href=path_BAG.absolute().resolve().as_posix())
item.add_asset(key="data", asset=asset)

item

<Item id=bag>

In [36]:
# Update the catalog with the new item
catalog = Catalog.from_file(path_root / 'catalog.json')
catalog.add_item(item)

<Link rel=item target=<Item id=bag>>

In [37]:
catalog.normalize_and_save(
    root_href=path_root.absolute().resolve().as_posix(),
    catalog_type=CatalogType.SELF_CONTAINED,
)

### Make a STAC catalog for KNMI data

In [38]:
path_knmi = Path('./data/knmi.zarr')
path_knmi.exists()

True

In [39]:
ds_knmi = xr.open_zarr(path_knmi)
ds_knmi

<xarray.Dataset>
Dimensions:   (station: 63, time: 965)
Coordinates:
  * station   (station) int64 201 203 204 205 206 207 ... 370 375 377 380 391
  * time      (time) datetime64[ns] 2008-01-01T01:00:00 ... 2008-02-13T19:00:00
Data variables: (12/33)
    DD        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    DR        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FF        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FH        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FX        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    HH        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    ...        ...
    TD        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    U         (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    VV        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    WW        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    Y         (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    YYYYMMDD  (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>

In [40]:
stations_geom = MultiPoint(
    [(x, y) for x, y in zip(ds_knmi["POS_X"], ds_knmi["POS_Y"])]
).__geo_interface__ # Convert to GeoJSON dict
stations_geom

{'type': 'MultiPoint',
 'coordinates': ((-4500.0, 707100.0),
  (15700.0, 487600.0),
  (37600.0, 588300.0),
  (55000.0, 825100.0),
  (65099.99999999999, 682000.0),
  (126700.0, 625300.0),
  (191800.0, 611700.0),
  (95900.0, 497600.0),
  (-5800.0, 651300.0),
  (71800.0, 548500.0),
  (197900.0, 672500.0),
  (89900.0, 461600.0),
  (98400.0, 497400.0),
  (110200.0, 556500.0),
  (114200.0, 548900.0),
  (110600.0, 763500.0),
  (114200.0, 481100.0),
  (123800.0, 583700.0),
  (140500.0, 516000.0),
  (127300.0, 517200.00000000006),
  (152200.0, 600400.0),
  (10300.0, 583300.0),
  (101700.0, 502100.0),
  (155900.0, 517700.00000000006),
  (140800.0, 456600.0),
  (140800.0, 456600.0),
  (154700.0, 545400.0),
  (164000.0, 496500.0),
  (179300.0, 581800.0),
  (188800.0, 523799.99999999994),
  (188200.0, 451800.0),
  (209000.0, 603000.0),
  (214200.0, 494300.0),
  (235000.0, 529600.0),
  (235100.0, 571300.0),
  (242000.0, 453900.0),
  (221900.0, 621200.0),
  (272700.0, 580000.0),
  (257600.00000000003

In [41]:
bbox = [
    ds_knmi["POS_X"].values.min(),
    ds_knmi["POS_Y"].values.min(),
    ds_knmi["POS_X"].values.max(),
    ds_knmi["POS_Y"].values.max(),
]
bbox

[-5800.0, 323900.0, 272700.0, 825100.0]

In [42]:
attrs_dict = {var_name: ds_knmi[var_name].attrs for var_name in ds_knmi.data_vars}
attrs_dict

{'DD': {'description': 'Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;'},
 'DR': {'description': 'Duur van de neerslag (in 0.1 uur) per uurvak;'},
 'FF': {'description': 'Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur;'},
 'FH': {'description': 'Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;'},
 'FX': {'description': 'Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak;'},
 'HH': {'description': 'tijd (HH=uur, UT.12 UT=13 MET, 14 MEZT. Uurvak 05 loopt van 05.00 UT tot 6.00 UT;'},
 'HOOGTE': {},
 'IX': {'description': 'Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2

In [43]:
# Create a new Item
item = Item(id='knmi',
            geometry=stations_geom,
            bbox=bbox,
            datetime=None,
            start_datetime=pd.Timestamp(ds_knmi['time'].min().values).to_pydatetime(),
            end_datetime=pd.Timestamp(ds_knmi['time'].min().values).to_pydatetime(),
            properties={"data_vars": list(ds_knmi.data_vars), 'geometry_types': 'Point'})

# Add the Datacube Extension to the Item
item.stac_extensions.append('datacube')

# Add the Datacube Extension properties to the Item
item.properties.update({'cube:variables': attrs_dict})

# Add path to the BAG dataset as an asset
asset = Asset(href=path_knmi.absolute().resolve().as_posix())
item.add_asset(key='data', asset=asset)

item

<Item id=knmi>

In [44]:
path_root = Path("./data/catalog/")
catalog = Catalog.from_file(path_root / 'catalog.json')
catalog

<Catalog id=contextual>

In [45]:
catalog.add_item(item)
catalog

<Catalog id=contextual>

In [46]:
catalog.normalize_and_save(path_root.absolute().resolve().as_posix(), catalog_type=CatalogType.SELF_CONTAINED)